<a href="https://colab.research.google.com/github/pgordin/OptDisc2024/blob/main/Grafy2_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from random import random
from copy import deepcopy

In [ ]:
class Graph:
    def __init__(self, graph=None):
        if graph is None:
            graph = {}
        self.graph = graph

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_array(cls, graph: np.array, nodes: list = None):
        if nodes is None:
            nodes = [*range(1, len(graph) + 1)]
        return cls.from_dict(
            cls._array_to_dict(graph, nodes)
        )

    @staticmethod
    def from_edges(filename: str, directed = 0):
        """
        Generates a graphs from a text file, where each line defines one edge.\n
        Filename is a file path.
        """
        graph = Graph()
        file = open(filename, "r")
        for line in file:
            words = line.strip().split()
            if len(words) == 1:
                graph.add_node(words[0])
            elif len(words) > 1: # more than two words -- we use first two
                if directed:
                    graph.add_arc([words[0], words[1]])
                else:
                    graph.add_edge([words[0], words[1]])
        file.close()
        return graph

    @staticmethod
    def random_graph(nodes_num: int, prob: float):
        """
        Generates a random graph provided a number of nodes and probability of generating an edge.
        """
        rand_graph = Graph()
        for i in range(1, nodes_num + 1):
            for j in range(1, i):
                if random() < prob:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(nodes_num: int):
        """
        Generates a cycle provided a number of nodes.
        """
        cycle = Graph()
        nodes = [*range(1, nodes_num + 1)]
        for i in nodes:
            cycle.add_edge([nodes[i-2], nodes[i-1]])
        cycle.graph = dict(sorted(cycle.graph.items()))
        return cycle

    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.\n
        Filename is a file path.
        """
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    def nodes(self) -> list:
        """
        Returns list of nodes of a graph.
        """
        return [*self.graph.keys()]

    def array(self) -> np.array:
        """
        Returns the graph in array form.
        """
        return self._dict_to_array(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
            res += "\n"
        return res

    def add_node(self, node):
        """
        Adds a node to a graph.
        """
        if node not in self.graph:
            self.graph[node] = []

    def del_node(self, node):
        """
        Recursively removes a node from a graph.
        """
        if node in self.graph:
            self.graph.pop(node)
            for key in [*self.graph.keys()]:
                if node in self.graph[key]:
                    self.graph[key].remove(node)

    def add_arc(self, arc: list):
        """
        Adds arc to a graph provided a list of nodes.
        """
        u, v = arc
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)

    def add_edge(self, edge: list):
        """
        Adds edge to a graph provided a list of nodes.
        """
        u, v = edge
        if u == v:
            raise ValueError("Pętla!")
        self.add_node(u)
        self.add_node(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)

    def _array_to_dict(arr: np.array, nodes: list) -> dict:
        """
        Converts a graph in array form to a graph in dict form.
        """
        res_dict = {}
        for i, node in enumerate(nodes):
            neighbours = [nodes[j] for j, edge in enumerate(arr[i]) if edge]
            res_dict[node] = neighbours
        return res_dict

    def _dict_to_array(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to a graph in array form.
        """
        n = len(_dict)
        nodes = [*_dict.keys()]
        res_arr = np.zeros(shape = (n, n), dtype=int)
        for u,v in [
            (nodes.index(u), nodes.index(v))
            for u, row in _dict.items() for v in row
        ]:
            res_arr[u][v] += 1
        return res_arr

    def Prufer(self):
      """
      Kod Prufera drzewa - zwrocony jako napis.
      Wymagane, aby graf byl drzewem.
      zwraca pusty napis dla drzew o mniej niz 3 wierzcholkach
      """
      tr = deepcopy(self.graph) #bedziemy psuc graf
      code = ""
      for i in range(len(self.graph)-2):
        for x in sorted(tr): # po kolei przegladam nieusuniete wierzcholki
          if len(tr[x])==1:   #najmniejszy lisc
            break
        v = tr[x][0]  #sasiad najmniejszego x
        code = code + f'{v} '
        tr[v].remove(x) #usuwam x z listy sasiadow v
        tr.pop(x)  #usuwam x z drzewa
      return code.strip()

    def tree_from_Prufer(code: str):
      """
      tworzy rzewo na podstawie kodu Prufera
      """
      tree = Graph()
      clist = [int(x) for x in code.strip().split()]  #kod zamieniamy na liste list
      n = len(clist) + 2  #liczba wierzcholkow
      vert = [v for v in range(1, n +1)] #lista licz od 1 do n
      for v in vert:
        tree.add_node(v)
      for i in range(n-2):
        for x in vert:
          if not x in clist: #najmniejszy lisc
            break
        v = clist.pop(0)    #usuwam pierwszy el. listy (sasiad x)
        tree.add_edge((x,v))
        vert.remove(x)
      tree.add_edge(vert)
      return tree

    def Connected_components(self):
        """
        Znajduje spójne składowe w grafie nieskierowanym.
        Jako wynik zwraca listę list wierzchołków.
        Uwaga: jako pierwszy element listy uzyskamy zbiór wszystkich wierzchołków grafu.
        """
        def DFS(u):
            """
            Przeszukiwanie w głąb
            """
            for w in self.graph[u]:
                if not w in VT[0]:             # w jeszcze nie odwiedzony
                    VT[0].add(w)               # w już odwiedzone
                    VT[-1].add(w)              # w ostatniej spójnej składowej
                    DFS(w)
        """
        VF - lista zbiorów VT[i] dla i>0 - lista wierzchołków spójnych składowych
        dla i=0 - lista wszystkich odwiedzonych wierzchołków
        """
        VT=[set([])]
        for v in self.graph:
            if v not in VT[0]:
                VT[0].add(v)
                VT.append(set([v]))          # zaczątek nowej spójnej składowej
                DFS(v)
        return VT

# Przykłady wykorzystania

In [ ]:
graph = Graph.random_graph(10, 1/2)
print(graph)
print("-------------------")
print(graph.Connected_components())


3: 2 6 7 8 10
2: 3 4 9
4: 1 2 5 9 10
1: 4 5 7 10
5: 1 4 7
6: 3 7 10
7: 1 3 5 6 9
8: 3 9 10
9: 2 4 7 8 10
10: 1 3 4 6 8 9

-------------------
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}]


In [ ]:
graph.add_edge((11, 12))
print(graph.Connected_components())

[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}, {11, 12}]


In [ ]:
tree = Graph.from_dict(
    {
    1: [2],
    2: [1, 3, 4],
    3: [2],
    4: [2]
    }
)
print(tree.Prufer())

2 2


In [ ]:
print(Graph.tree_from_Prufer("2 2"))

1: 2
2: 1 3 4
3: 2
4: 2



In [ ]:
tree2 = Graph.tree_from_Prufer("1 2 7 3 4")
print(tree2)
print("--------------------------")
print(tree2.Prufer())

1: 5 2
2: 1 7
3: 6 4
4: 3 7
5: 1
6: 3
7: 2 4

--------------------------
1 2 7 3 4


In [ ]:
tree3 = Graph.tree_from_Prufer("1 1 7 7 6 2")
print(tree3)
print("--------------------------")
print(tree3.Prufer())

1: 3 4 7
2: 6 8
3: 1
4: 1
5: 7
6: 7 2
7: 1 5 6
8: 2

--------------------------
1 1 7 7 6 2


In [ ]:
tree4 = Graph.tree_from_Prufer(" ")
print(tree4)
print("--------------------------")
print(tree4.Prufer())

1: 2
2: 1

--------------------------



In [ ]:
graph = Graph.from_dict(
    {
        'a': ['b', 'c'],
        'b': ['a', 'd', 'e'],
        'c': ['a', 'f'],
        'd': ['b', 'f'],
        'e': ['b'],
        'f': ['c', 'd'],
        'g': [],
    }
)

graph_array = graph.array()
graph_nodes = graph.nodes()

print(graph)
print(graph_array)
print(graph_nodes)

a: b c
b: a d e
c: a f
d: b f
e: b
f: c d
g:

[[0 1 1 0 0 0 0]
 [1 0 0 1 1 0 0]
 [1 0 0 0 0 1 0]
 [0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0]
 [0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0]]
['a', 'b', 'c', 'd', 'e', 'f', 'g']


In [ ]:
random_graph = Graph.random_graph(10, 1/3)
print(random_graph)

4: 3 7
3: 4 9
6: 1 5 10
1: 6 7 10
5: 6 7 8 9 10
7: 1 4 5 8 9
8: 2 5 7 10
2: 8 10
9: 3 5 7
10: 1 2 5 6 8



In [ ]:
cycle = Graph.cycle(10)
print(cycle)

1: 10 2
2: 1 3
3: 2 4
4: 3 5
5: 4 6
6: 5 7
7: 6 8
8: 7 9
9: 8 10
10: 1 9



In [ ]:
%%writefile lista.txt
A B
B C
B D
D C
E
F

Writing lista.txt


In [ ]:
file_graph = Graph.from_edges("./lista.txt")
print(file_graph)

A: B
B: A C D
C: B D
D: B C
E:
F:



In [ ]:
file_graph.add_edge(["E", "F"])
file_graph.to_neighbourlist("lista1.txt")

In [ ]:
%cat lista1.txt

A: B
B: A C D
C: B D
D: B C
E: F
F: E


#ZADANIA

### Zad1.
Napisać funkcję preorder(v) i postorder(v) które wypiszą drzewa (np. wygenerowane z kodu Prüfera) w porządku preorder lub (odpowiednio) postorder, począwszy od wierzchołka v.

In [ ]:
def preorder(tree, v, visited=None, pre=None):
    """
    Funkcja wypisująca drzewa w porządku preorder
    - tree to obiekt reprezentujący drzewo
    - v to wierzchołek, od którego rozpoczynamy przechodzenie drzewa
    - visited jest opcjonalnym parametrem, przechowującym zbiór odwiedzonych wierzchołków
    - pre to lista, która będzie przechowywać wierzchołki drzewa w kolejności preorder
    """
    if visited==None:
        visited = set() #tworzymy zbior visited
    if pre==None:
        pre=[]          #tworzymy liste pre
    visited.add(v)
    pre.append(v)       #dodanie do listy odwiedzonego wierzcholka (przed znalezieniem jego sasiadow)
    for neighbor in tree.graph[v]:  #dla sąsiadów danego wierzchołka
        if neighbor not in visited:  #jesli sąsiad nie był odwiedzony
            preorder(tree,neighbor, visited, pre)  #powtarzamy cala funkcje traktujac dany wierzcholek jako poczatek aktualnego drzewa
    return pre  #zwracamy wierzcholki w kolejnosci


In [ ]:
tree = Graph.from_dict(
    {
    1: [2],
    2: [1, 3, 4],
    3: [2,5],
    4: [2],
    5: [1,4]
    }
)

In [ ]:
print(preorder(tree,1))

[1, 2, 3, 5, 4]


In [ ]:
def postorder(tree, v, visited=None, post=None):
    """
    Funkcja wypisująca drzewa w porządku postorder
    - tree to obiekt reprezentujący drzewo
    - v to wierzchołek, od którego rozpoczynamy przechodzenie drzewa
    - visited jest opcjonalnym parametrem, przechowującym zbiór odwiedzonych wierzchołków
    - post to lista, która będzie przechowywać wierzchołki drzewa w kolejności postorder
    """
    if visited is None:
        visited = set() #tworzenie zbioru visited
    if post is None:
        post=[]         #tworzenie listy post
    visited.add(v)      #dodanie wierzcholka do odwiedzonych
    for neighbor in tree.graph[v]:  #dla sasiada danego wierzcholka
        if neighbor not in visited:   #jesli sasiad nie byl odwiedzony
            postorder(tree, neighbor, visited, post)   ##powtarzamy cala funkcje traktujac dany wierzcholek jako poczatek aktualnego drzewa
    post.append(v)  #dodanie do listy odwiedzonego wierzcholka (po znalezieniu jego sasiadow)
    return post

In [ ]:
tree = Graph.from_dict(
    {
    1: [2],
    2: [1, 3, 4],
    3: [2,5],
    4: [2],
    5: [1,4]
    }
)

In [ ]:
print(postorder(tree,1))

[4, 5, 3, 2, 1]


In [ ]:
print(preorder(tree,1))

[1, 2, 3, 5, 4]


###zad2
Pokazana na zajęciach funkcja ConnectedComponents zwraca listę zbiorów wierzchołków. Napisać funkcję ConnectedComponentsGraphs(), która zwróci listę grafów — spójnych składowych grafu podanego jako jej parametr. Można (warto) wykorzystać (gotową) funkcję ConnectedComponents.

In [ ]:
def Connected_components_graphs(graph):
    VT=graph.Connected_components()
    noCc=len(VT)-1  #liczba spojnych skladowych
    graphs=[]  #lista do przechowywania grafow reprezentujacych spojne skladowe
    for i in range(noCc):  #iterujemy przez wszystkie spojne skladowe
        graph_i={}    #dla każdej spójnej składowej tworzymy nowy graf graph_i, który jest podgrafem oryginalnego grafu,
        for v in VT[i+1]:     #zawierającym tylko wierzchołki z bieżącej spójnej składowej
            graph_i[v]=graph.graph[v]
        graphs.append(graph_i)  #dodajemy stworzone grafy do listy
    return graphs

In [ ]:
graph = Graph.from_dict(
    {
        'a': ['b', 'c'],
        'b': ['a', 'd', 'e'],
        'c': ['a', 'f'],
        'd': ['b', 'f'],
        'e': ['b'],
        'f': ['c', 'd'],
        'g': [],
    }
)
print(graph)

a: b c
b: a d e
c: a f
d: b f
e: b
f: c d
g:



In [ ]:
Connected_components_graphs(graph)

[{'c': ['a', 'f'],
  'd': ['b', 'f'],
  'f': ['c', 'd'],
  'a': ['b', 'c'],
  'e': ['b'],
  'b': ['a', 'd', 'e']},
 {'g': []}]

### zad3.
Napisać funkcję random_bipartite_graph(n, p) która wygeneruje dwudzielny graf losowy o 2n wierzchołkach (podgraf grafu Kn,n ) w którym każda możliwa para wierzchołków jest połączona krawędzią niezależnie, z p-stwem p.

In [ ]:
def random_bipartite_graph(n,p):
    v1=[]  #pusta lista wierzcholkow
    v2=[]  #pusta lista wierzcholkow
    graph={}
    for i in range(1,n+1):
        v1.append(i)    #generujemy wierzcholki od 1 do n i zapisujemy w v1 dana wartosc
        v2.append(i+n)  #generujemy wierzcholki od 1+n do 2n i zapisujemy w v1 dana wartosc

    for vertex1 in v1:
        values=[]
        for vertex2 in v2:
            if random()<p:    #pętla sprawdza, czy dla danego połączenia między wierzchołkiem z v1 a wierzchołkiem z v2 powinna zostać dodana krawędź
                values.append(vertex2)  #jesli wylosowana liczba z (0,1)<p dodaje krawedz
        graph[vertex1]=values   #Sąsiedzi danego wierzchołka są przechowywani w liście values,
                                #a następnie ta lista jest przypisywana jako wartość dla klucza będącego numerem wierzchołka z v1 w słowniku graph.

    return Graph.from_dict(graph)

In [ ]:
bipartite_graph=random_bipartite_graph(3,1/2)
print(bipartite_graph)

1: 4 5 6
2: 4
3: 5 6

